In [5]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from typing import List, Tuple, Optional
from dataclasses import dataclass
from scipy.linalg import eigh

In [6]:
class Shape3D:
    def __init__(self, points: np.ndarray):
        """
        Initialize with nx3 array of points
        """
        self.points = points
        self.center = None
        self.centered_points = None
        self.covariance = None
        self.eigenvalues = None
        self.eigenvectors = None
        self.principal_axes = None
        
    def compute_pca(self):
        """
        Perform PCA analysis on the shape
        """
        # Compute center of mass
        self.center = np.mean(self.points, axis=0)
        
        # Center the points
        self.centered_points = self.points - self.center
        
        # Compute covariance matrix
        self.covariance = np.cov(self.centered_points.T)
        
        # Compute eigenvalues and eigenvectors
        self.eigenvalues, self.eigenvectors = eigh(self.covariance)
        
        # Sort by eigenvalues in descending order
        idx = np.argsort(self.eigenvalues)[::-1]
        self.eigenvalues = self.eigenvalues[idx]
        self.eigenvectors = self.eigenvectors[:, idx]
        
        # Compute principal axes (scaled by eigenvalues for visualization)
        scale = np.sqrt(self.eigenvalues)
        self.principal_axes = self.eigenvectors * scale[:, np.newaxis]
    
    def get_transformation_matrix(self) -> np.ndarray:
        """
        Get transformation matrix for aligning shape with principal axes
        """
        return self.eigenvectors.T
    
    def transform_to_principal_axes(self) -> np.ndarray:
        """
        Transform points to align with principal axes
        """
        return np.dot(self.centered_points, self.eigenvectors)

class ShapeGenerator:
    @staticmethod
    def generate_ellipsoid(n_points: int = 1000, scales: Tuple[float, float, float] = (3, 2, 1), 
                          rotation: Optional[np.ndarray] = None, noise: float = 0.1) -> np.ndarray:
        """
        Generate ellipsoid points with optional rotation and noise
        """
        # Generate spherical points
        phi = np.random.uniform(0, 2*np.pi, n_points)
        theta = np.random.uniform(0, np.pi, n_points)
        
        # Convert to Cartesian coordinates
        x = np.sin(theta) * np.cos(phi)
        y = np.sin(theta) * np.sin(phi)
        z = np.cos(theta)
        
        points = np.vstack((x, y, z)).T
        
        # Scale to create ellipsoid
        points = points * scales
        
        # Add random noise
        points += np.random.normal(0, noise, points.shape)
        
        # Apply rotation if specified
        if rotation is not None:
            points = np.dot(points, rotation)
            
        return points

class PCAVisualization:
    def __init__(self):
        self.fig = None
        self.shape = None
        self.steps = []
        self.current_step = 0
        
    def create_shape(self):
        """
        Create a sample shape for analysis
        """
        # Create rotation matrix (45 degrees around z-axis)
        theta = np.pi/4
        rotation = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta), np.cos(theta), 0],
            [0, 0, 1]
        ])
        
        # Generate rotated ellipsoid
        points = ShapeGenerator.generate_ellipsoid(
            n_points=500,
            scales=(3, 2, 1),
            rotation=rotation,
            noise=0.5
        )
        
        self.shape = Shape3D(points)
        self.shape.compute_pca()
        
        # Store visualization steps
        self.steps = [
            self._create_initial_step(),
            self._create_centered_step(),
            self._create_eigenvectors_step(),
            self._create_transformed_step()
        ]
    
    def _create_initial_step(self) -> dict:
        """Create initial visualization step"""
        return {
            "points": self.shape.points,
            "center": None,
            "axes": None,
            "title": "Original Shape"
        }
    
    def _create_centered_step(self) -> dict:
        """Create centered points visualization step"""
        return {
            "points": self.shape.centered_points,
            "center": np.zeros(3),
            "axes": None,
            "title": "Centered Shape"
        }
    
    def _create_eigenvectors_step(self) -> dict:
        """Create step showing principal axes"""
        return {
            "points": self.shape.centered_points,
            "center": np.zeros(3),
            "axes": self.shape.principal_axes,
            "title": "Principal Axes"
        }
    
    def _create_transformed_step(self) -> dict:
        """Create transformed shape visualization step"""
        return {
            "points": self.shape.transform_to_principal_axes(),
            "center": np.zeros(3),
            "axes": np.array([[1,0,0], [0,1,0], [0,0,1]]) * np.sqrt(self.shape.eigenvalues)[:, np.newaxis],
            "title": "Transformed Shape"
        }
    
    def update_figure(self, step_data: dict) -> None:
        """Update figure with current step data"""
        points = step_data["points"]
        center = step_data["center"]
        axes = step_data["axes"]
        
        self.fig = go.Figure()
        
        # Plot points
        self.fig.add_trace(go.Scatter3d(
            x=points[:, 0],
            y=points[:, 1],
            z=points[:, 2],
            mode='markers',
            marker=dict(
                size=3,
                color='blue',
                opacity=0.6
            ),
            name='Points'
        ))
        
        # Plot center if exists
        if center is not None:
            self.fig.add_trace(go.Scatter3d(
                x=[center[0]],
                y=[center[1]],
                z=[center[2]],
                mode='markers',
                marker=dict(
                    size=8,
                    color='red',
                    symbol='cross'
                ),
                name='Center'
            ))
        
        # Plot principal axes if exist
        if axes is not None:
            colors = ['red', 'green', 'blue']
            for i, (axis, color) in enumerate(zip(axes.T, colors)):
                if center is not None:
                    start = center
                else:
                    start = np.zeros(3)
                    
                end = start + axis
                
                self.fig.add_trace(go.Scatter3d(
                    x=[start[0], end[0]],
                    y=[start[1], end[1]],
                    z=[start[2], end[2]],
                    mode='lines',
                    line=dict(color=color, width=5),
                    name=f'Principal Axis {i+1}'
                ))
        
        # Update layout
        self.fig.update_layout(
            scene=dict(
                aspectmode='data',
                camera=dict(
                    up=dict(x=0, y=1, z=0),
                    center=dict(x=0, y=0, z=0),
                    eye=dict(x=1.5, y=1.5, z=1.5)
                )
            ),
            title=step_data["title"],
            height=800,
            showlegend=True
        )

def create_interactive_visualization():
    """Create interactive visualization with controls"""
    vis = PCAVisualization()
    vis.create_shape()
    
    play_button = widgets.Play(
        value=0,
        min=0,
        max=len(vis.steps) - 1,
        step=1,
        interval=1500,
        description="Play",
        disabled=False
    )
    
    slider = widgets.IntSlider(
        value=0,
        min=0,
        max=len(vis.steps) - 1,
        step=1,
        description='Step:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )
    
    # Add information widget
    info_text = widgets.HTML(
        value="""<h3>PCA Steps:</h3>
        <ol>
            <li>Original Shape</li>
            <li>Center Shape</li>
            <li>Find Principal Axes</li>
            <li>Transform Shape</li>
        </ol>"""
    )
    
    reset_button = widgets.Button(
        description='Reset',
        disabled=False,
        button_style='warning',
        tooltip='Reset visualization'
    )
    
    widgets.jslink((play_button, 'value'), (slider, 'value'))
    
    def update(change):
        step_index = change['new']
        vis.update_figure(vis.steps[step_index])
        display(vis.fig)
    
    def reset(_):
        slider.value = 0
        play_button.value = 0
    
    slider.observe(update, names='value')
    reset_button.on_click(reset)
    
    # Create layout
    controls = widgets.VBox([
        widgets.HBox([play_button, slider, reset_button]),
        info_text
    ])
    
    vis.update_figure(vis.steps[0])
    display(controls)
    display(vis.fig)
    
    return vis

In [7]:
visualization = create_interactive_visualization()

In [8]:
def validate_pca(shape: Shape3D):
    """
    Validates PCA results
    """
    # Compute and print eigenvalues
    print("Eigenvalues (variance explained by each principal component):")
    total_variance = sum(shape.eigenvalues)
    for i, eigenvalue in enumerate(shape.eigenvalues, 1):
        percentage = (eigenvalue / total_variance) * 100
        print(f"PC{i}: {eigenvalue:.4f} ({percentage:.2f}% of total variance)")
    
    # Check orthogonality of eigenvectors
    print("\nEigenvector orthogonality check (should be close to 0):")
    for i in range(3):
        for j in range(i+1, 3):
            dot_product = np.dot(shape.eigenvectors[:, i], shape.eigenvectors[:, j])
            print(f"PC{i+1} · PC{j+1} = {dot_product:.10f}")
    
    # Check correlation in transformed data
    transformed_points = shape.transform_to_principal_axes()
    correlation = np.corrcoef(transformed_points.T)
    print("\nCorrelation matrix of transformed data (should be close to diagonal):")
    print(correlation)
    
    # Reconstruction error
    reconstructed_points = np.dot(transformed_points, shape.eigenvectors.T) + shape.center
    error = np.mean(np.square(reconstructed_points - shape.points))
    print(f"\nReconstruction error: {error:.10f}")
    
    return {
        'eigenvalues': shape.eigenvalues,
        'correlation': correlation,
        'error': error
    }

metrics = validate_pca(visualization.shape)

Eigenvalues (variance explained by each principal component):
PC1: 2.2982 (52.38% of total variance)
PC2: 1.3122 (29.91% of total variance)
PC3: 0.7770 (17.71% of total variance)

Eigenvector orthogonality check (should be close to 0):
PC1 · PC2 = -0.0000000000
PC1 · PC3 = 0.0000000000
PC2 · PC3 = -0.0000000000

Correlation matrix of transformed data (should be close to diagonal):
[[ 1.00000000e+00 -6.50327964e-16 -7.99168174e-18]
 [-6.50327964e-16  1.00000000e+00 -1.41013160e-17]
 [-7.99168174e-18 -1.41013160e-17  1.00000000e+00]]

Reconstruction error: 0.0000000000
